# ショアのアルゴリズムの実装
シミュレーターを用いて実装する

In [1]:
from quantum_simulater import * # シミュレーター
from inv_QFT import *           # 逆量子フーリエ変換
from fractions import Fraction  # 有理数近似に用いる
import re                       # 分母を取り出すときに用いる

### やりたいことの確認
1. 因数分解したい数Nに対して、Nと互いに素でありNより小さい自然数xを定める。  
2. そのNとxを使って、ユニタリ行列$U_{x}=\sum_{y=0}^{2^{n}-1} |xy (mod N)><y|$ を行列表現する。($2^{n-1}-1<N-1<=2^{n}-1$)  
3. その$U_{x}$の固有値の位相$\psi$を推定する。  
4. $s/t(=\psi)$に対して連分数展開を行い位数$t$を求める。
5. $t$が偶数なら$gcd(x^{t/2}-1,N)$か$gcd(x^{t/2}+1,N)$がNの因数となる。

# xを定める
Nに対する適当なxと位数rを求める関数  
条件を満たすxが見つかると一度関数を中断して値を返すyieldが使われている  

In [2]:
def find_x(N):
    for x in range(2,int(N)):
        if math.gcd(x,int(N))==1:
            yield x
        else:
            continue

# Uxを生成
$U_{x}$を行列表現する関数

In [3]:
def Ux(N,x):
    # Nに対するnを求める
    n = math.ceil(math.log(N, 2))
    
    Ux = np.zeros((2**n,2**n))
    
    for y in range(2**n):
        ket = np.zeros((2**n,1))
        ket[(x*y)%N,0] = 1
        bra = np.zeros((1,2**n))
        bra[0,y] = 1
        Ux = Ux + np.kron(ket,bra)
    
    return Ux

# 位数推定
解答レジスタの初期状態は|0>  
問題レジスタの初期状態は|1>

In [4]:
def get_fac(N,x):
    # Nに対するnを求める
    n = math.ceil(math.log(N, 2))

    # 初期状態の設定
    qubits = Qubits(2*n)     # 解答レジスタ + 問題レジスタで 2nビット
    qubits.set_bit(2*n-1, 1) # 解答レジスタの初期状態 |0> 問題レジスタの初期状態 |1>
    
    # Hゲートを第0～n-1ビットに作用させる
    for i in range(n):
        qubits.apply(H(2*n,i))
    
    # CUkを作用させる
    Uk_list = [Ux(N,x)]
    for k in range(n):
        Uk_list.append(Uk_list[k] @ Uk_list[k])      # k番目の成分は Ux^{2^{k}} 
        qubits.apply(CU(2*n, n-1-k, n, Uk_list[k]))  # 第n-1-k番目をを制御ビットとし Ux^{2^{k}} を作用させる
    
    # 第0～n-1ビットに対し逆量子フーリエ変換
    qubits.apply(tensor(invQFT(n), I(n)))
    
    # ビットの状態を観測する (本当は第0～n-1ビットを観測するだけでよい)
    qubits.measure()
    
    # 第0～n-1ビットを受け取り、その二進少数を十進少数に変換
    Q = " + ".join(
        ("{:0" + str(qubits.n_bits) + "b}").format(i)    # 量子状態を文字列として得る
        for i, amp in enumerate(qubits._amp) if amp==1
    )
    Phase = 0
    for i in range(n):
        if Q[i] == "1":
            Phase += (1/2)**(i+1)
    
    # 位相を分数表示にしてその分母を位数候補として取得する
    if Phase == 0:
        RANK = 0
    else:
        str_frac = str(Fraction(Phase))                 # 分数表示
        pattern  = "(.*)/(.*)"                          # " / " で区切る
        RANK     = int(re.split(pattern, str_frac)[2])  # 区切った 2番目が分母

    # 位数から因数候補を求め、返す
    if RANK == 0:
        return 1
    else:
        fac1 = math.gcd(x**int((RANK/2))-1, N)
        fac2 = math.gcd(x**int((RANK/2))+1, N)
        if   N%fac1 == 0 and N != fac1: return fac1
        elif N%fac2 == 0 and N != fac2: return fac2
        else: return 1

# ショアのアルゴリズム(二つに分解する)
後は自然数Nを受け取り因数分解する関数を作れば良い

In [5]:
def Q_shor(N):
    while(True):
        for x in find_x(N):
            fac = get_fac(N,x)
            if N%fac == 0 and fac != 1:
                return fac, int(N/fac)

In [6]:
Q_shor(15)   # 素数入れると永遠続くので注意

(3, 5)

40くらいで結構怖い...  
Jupyter Notebook ではメニューのKernelからInterruptを選ぶと実行を停止できる。  